In [ ]:
# ---------------------------------------------------------
# 1. Imports & Setup
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt



# ---------------------------------------------------------
# 2. Load & Basic Cleaning
# ---------------------------------------------------------
# Adjust the file path as needed
df = pd.read_csv("Titanic-Dataset.csv")

# Keep only the needed columns
cols_needed = ["Survived", "Pclass", "Sex", "Age", "Fare"]
df = df[cols_needed].copy()

# Drop rows with missing values in these columns
df.dropna(subset=cols_needed, inplace=True)

# Convert 'Sex' to numeric: 1 = male, 0 = female
df["Sex"] = (df["Sex"] == "male").astype(int)

# One-hot encode 'Pclass', but drop_first=True to avoid perfect collinearity
df = pd.get_dummies(df, columns=["Pclass"], prefix="Pclass", drop_first=True)
# This will create (for a 3-class variable) only 2 columns, e.g. Pclass_2 and Pclass_3.
# Pclass_1 is implied as the baseline.

# Force numeric dtypes (just in case)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop rows that might have become NaN
df.dropna(inplace=True)

# Check final shape and unique values
print("DataFrame shape after cleaning:", df.shape)
print("\nUnique values per column:")
print(df.nunique())

# If any column has only 1 unique value, it won't help the model.
# For example:
# for col in df.columns:
#     if df[col].nunique() == 1:
#         print(f"Column '{col}' is constant and will be removed.")
#         df.drop(columns=[col], inplace=True)

# ---------------------------------------------------------
# 3. Define X (features) and y (target)
# ---------------------------------------------------------
# Target
y = df["Survived"].astype(int).values

# Example features:
#   Sex, Age, Fare, Pclass_2, Pclass_3
X_cols = ["Sex", "Age", "Fare", "Pclass_2", "Pclass_3"]
for c in X_cols:
    if c not in df.columns:
        raise ValueError(f"Missing expected column '{c}' in DataFrame.")

X = df[X_cols].astype(float).values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)
print("Feature matrix dtype:", X.dtype, "| Target vector dtype:", y.dtype)

# ---------------------------------------------------------
# 4. Build & Sample from Bayesian Logistic Model
# ---------------------------------------------------------
with pm.Model() as logistic_model:
    # Priors on coefficients
    betas = pm.Normal("betas", mu=0, sigma=1, shape=X.shape[1])
    
    # Linear predictor
    eta = pm.math.dot(X, betas)
    
    # Logistic transformation
    p = pm.Deterministic("p", pm.math.sigmoid(eta))
    
    # Bernoulli likelihood
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)
    
    # MCMC sampling
    trace = pm.sample(
        draws=2000,       # Posterior draws
        tune=1000,        # Warm-up
        target_accept=0.9,
        random_seed=42
    )

# ---------------------------------------------------------
# 5. Posterior Summaries & Diagnostics
# ---------------------------------------------------------
summary = az.summary(trace, var_names=["betas"], round_to=2)
print("\nPosterior Summary for 'betas':")
display(summary)

# Trace plot
az.plot_trace(trace, var_names=["betas"])
plt.show()




DataFrame shape after cleaning: (714, 6)

Unique values per column:
Survived      2
Sex           2
Age          88
Fare        220
Pclass_2      2
Pclass_3      2
dtype: int64

Feature matrix shape: (714, 5)
Target vector shape: (714,)
Feature matrix dtype: float64 | Target vector dtype: int64


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas]


In [16]:
# -----------------------------------------------------
# 1. Imports & Setup
# -----------------------------------------------------
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# Optional: nicer plot style
az.style.use("arviz-darkgrid")

# -----------------------------------------------------
# 2. Load Data & Inspect Columns
# -----------------------------------------------------
df = pd.read_csv("student-mat.csv", sep=";")

print("=== Columns in the dataset ===")
print(df.columns.tolist())


# Check if any columns are missing
missing_cols = [c for c in cols_needed if c not in df.columns]
if missing_cols:
    print(f"\nWARNING: The following columns are missing: {missing_cols}")
    print("Adjust your 'cols_needed' to match the actual column names in your CSV.")
else:
    print("\nAll required columns found!")

# -----------------------------------------------------
# 3. Subset & Clean Data
# -----------------------------------------------------


# Optional: downsample to 100 rows for faster demonstration
df = df.sample(n=100, random_state=42).reset_index(drop=True)

print("\n=== Final shape after subsetting & downsampling ===")
print(df.shape)
print(df.head())

# -----------------------------------------------------
# 4. Define Outcomes (Y) and Predictors (X)
# -----------------------------------------------------
# Two continuous outcomes
Y = df["G1", "G2"].values  # shape: (n, 2)

# Four predictors
X_cols = ["studytime", "failures", "age", "absences"]
X = df[X_cols].values        # shape: (n, 4)

print("\nY shape:", Y.shape)  # (100, 2) if you downsampled
print("X shape:", X.shape)    # (100, 4)

# -----------------------------------------------------
# 5. Build & Sample from Bayesian Multivariate Model
# -----------------------------------------------------
# We'll assume Y ~ MvNormal(mu, Sigma), where:
#   mu = X * betas + intercept
#   betas: shape (4, 2)
#   intercept: shape (2,)
#   Sigma is built from an LKJ prior for correlation + scale for each dimension

with pm.Model() as multivariate_model:
    # Regression coefficients: (p, m)
    betas = pm.Normal("betas", mu=0, sigma=1, shape=(X.shape[1], Y.shape[1]))
    
    # Intercept: one per outcome dimension
    intercept = pm.Normal("intercept", mu=0, sigma=1, shape=Y.shape[1])
    
    # Linear predictor
    mu = pm.math.dot(X, betas) + intercept  # shape (n, 2)
    
    # Covariance structure
    # sigma = marginal std dev for each outcome
    sigma = pm.Exponential("sigma", 1.0, shape=Y.shape[1])
    # LKJ prior for correlation
    Lcorr = pm.LKJCholeskyCov("Lcorr", n=Y.shape[1], eta=2.0, 
                              sd_dist=pm.Exponential.dist(1.0))
    
    # Construct the full Cholesky factor
    chol = pm.Deterministic("chol", pm.diag(sigma) @ Lcorr)
    
    # Multivariate Normal likelihood
    y_obs = pm.MvNormal("y_obs", mu=mu, chol=chol, observed=Y)
    
    # MCMC sampling
    trace = pm.sample(
        draws=2000, 
        tune=1000, 
        target_accept=0.9, 
        random_seed=42
    )

# -----------------------------------------------------
# 6. Posterior Summary & Diagnostics
# -----------------------------------------------------
summary = az.summary(trace, var_names=["betas", "intercept", "sigma", "Lcorr"], round_to=2)
print("\n=== Posterior Summary ===")
display(summary)

# Trace plots
az.plot_trace(trace, var_names=["betas", "intercept", "sigma"])
plt.show()

# (Optional) Pair plot for correlation parameters
az.plot_pair(trace, var_names=["sigma", "Lcorr"], divergences=True, kind="kde")
plt.show()


=== Columns in the dataset ===
['school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3']

Adjust your 'cols_needed' to match the actual column names in your CSV.

=== Final shape after subsetting & downsampling ===
(100, 1)
  school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0  GP,M,17,U,GT3,T,2,1,other,other,home,mother,2,...                                                                                                                                                                                 
1  MS,M,18,R,LE3,T,1,2,at_home,services,other,fat...                                                                                          

KeyError: ('G1', 'G2')

In [1]:
# =========================================================
# Complete Bayesian Multivariate Classification in One Code Chunk
# =========================================================
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# 1. Load Data
df = pd.read_csv("Employee.csv")

# Example columns:
#   - Two binary outcomes: "Attrition" (0/1), "Overtime" (0/1)
#   - Predictors: "Age", "MonthlyIncome", "YearsAtCompany"
binary_outcomes = ["Attrition", "Overtime"]
predictors = ["Age", "MonthlyIncome", "YearsAtCompany"]

# 2. Subset & Clean
all_needed = binary_outcomes + predictors
df = df[all_needed].dropna()

# 3. Define X (predictors) and Y (binary outcomes)
Y = df[binary_outcomes].values  # shape (n, 2)
X = df[predictors].values      # shape (n, 3)

# (Optional) Downsample for demonstration
n_desired = 200  # choose how many rows to keep
if X.shape[0] > n_desired:
    idx = np.random.choice(X.shape[0], n_desired, replace=False)
    X = X[idx]
    Y = Y[idx]

# (Optional) Scale numeric predictors
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 4. PyMC Model: Latent-Variable Multivariate Classification
az.style.use("arviz-darkgrid")
n, m = Y.shape
p = X.shape[1]

with pm.Model() as multi_label_model:
    # Coefficients: shape=(p, m)
    betas = pm.Normal("betas", mu=0, sigma=1, shape=(p, m))
    
    # Intercepts: shape=(m,)
    intercept = pm.Normal("intercept", mu=0, sigma=1, shape=m)
    
    # Covariance via LKJ prior
    sigma = pm.Exponential("sigma", 1.0, shape=m)
    Lcorr = pm.LKJCholeskyCov("Lcorr", n=m, eta=2.0,
                              sd_dist=pm.Exponential.dist(1.0))
    chol = pm.Deterministic("chol", pm.diag(sigma) @ Lcorr)
    
    # Mean of latent variable: mu = X * betas + intercept
    mu = pm.math.dot(X, betas) + intercept  # shape (n, m)
    
    # Latent variable z ~ MvNormal
    z = pm.MvNormal("z", mu=mu, chol=chol, shape=(n, m))
    
    # Logistic link => p = sigmoid(z)
    p_ = pm.Deterministic("p", pm.math.sigmoid(z))
    
    # Observed binary outcomes
    y_obs = pm.Bernoulli("y_obs", p=p_, observed=Y)
    
    # MCMC sampling
    trace = pm.sample(draws=2000, tune=1000, target_accept=0.9, random_seed=42)

# 5. Posterior Summary & Diagnostics
summary = az.summary(trace, var_names=["betas", "intercept", "sigma", "Lcorr"], round_to=2)
print("\n=== Posterior Summary ===")
print(summary)

az.plot_trace(trace, var_names=["betas", "intercept", "sigma"])
plt.show()

az.plot_pair(trace, var_names=["sigma", "Lcorr"], divergences=True, kind="kde")
plt.show()

KeyError: "['Attrition', 'Overtime', 'MonthlyIncome', 'YearsAtCompany'] not in index"